In [1]:
import calendar
import pandas as pd
import panel as pn

from datetime import date, timedelta
from sqlalchemy import create_engine

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
today

datetime.date(2022, 1, 22)

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trade name qty target active reason market'.split()

In [3]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite)

,id,trade,name,qty,price,active,reason,market
0,692,B,AEONTS,1200,180,0,3L,SET100


### After call Ord-Log.bat which calculates spreads between target price and SET price.

In [4]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name
#col_names   = ['trade','name','spreads','reason','market','qty','target_price','set_price','chg_amt','chg_pct','active']
df_log = pd.read_csv(input_file, sep=',',  index_col=None)
df_log

,trade,name,spd,reason,market,qty,target,current,change,percent,active
0,B,AEONTS,-5,3L,SET100,1200,180.00,182.50,-0.50,-0.27%,0
1,B,BGC,-7,DOS,SET,10000,10.00,10.70,0.10,+0.94%,0
2,B,NOBLE,-9,RD15%,SET,21000,5.85,6.30,0.00,0.00%,0
3,B,PTT,-9,RD05%,SET50,3000,36.50,38.75,-0.75,-1.90%,0
4,B,PTTGC,-12,RD10pct,SET50,1200,54.25,57.25,-0.50,-0.87%,0
5,B,RATCH,-5,RD05%,SET50,3000,42.75,44.00,0.00,0.00%,0
6,B,RCL,-6,RD05pct,SET100,1500,41.50,43.00,-2.00,-4.44%,0
7,B,ROJNA,-5,6L,SET,18000,6.30,6.55,0.00,0.00%,0
8,B,SCC,-4,RD05pct,SET50,300,374.00,378.00,-2.00,-0.53%,0
9,B,STA,-5,RD15%,SET100,2500,28.00,29.25,0.00,0.00%,0


In [5]:
def calc(vals):
    spreads = vals
    if abs(spreads) < 10:
        return 1
    else:
        return 0

In [6]:
df_log['active'] = df_log['spd'].apply(calc)

In [7]:
df_log.groupby('active')['trade'].count()

active
0     9
1    16
Name: trade, dtype: int64

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

25

In [9]:
df_log.dtypes

trade       object
name        object
spd          int64
reason      object
market      object
qty          int64
target     float64
current    float64
change     float64
percent     object
active       int64
dtype: object

In [10]:
final = df_log[colt]
final

,trade,name,qty,target,active,reason,market
0,B,AEONTS,1200,180.00,1,3L,SET100
1,B,BGC,10000,10.00,1,DOS,SET
2,B,NOBLE,21000,5.85,1,RD15%,SET
3,B,PTT,3000,36.50,1,RD05%,SET50
4,B,PTTGC,1200,54.25,0,RD10pct,SET50
5,B,RATCH,3000,42.75,1,RD05%,SET50
6,B,RCL,1500,41.50,1,RD05pct,SET100
7,B,ROJNA,18000,6.30,1,6L,SET
8,B,SCC,300,374.00,1,RD05pct,SET50
9,B,STA,2500,28.00,1,RD15%,SET100


In [11]:
rcds = final.values.tolist()
rcds

[['B', 'AEONTS', 1200, 180.0, 1, '3L', 'SET100'],
 ['B', 'BGC', 10000, 10.0, 1, 'DOS', 'SET'],
 ['B', 'NOBLE', 21000, 5.85, 1, 'RD15%', 'SET'],
 ['B', 'PTT', 3000, 36.5, 1, 'RD05%', 'SET50'],
 ['B', 'PTTGC', 1200, 54.25, 0, 'RD10pct', 'SET50'],
 ['B', 'RATCH', 3000, 42.75, 1, 'RD05%', 'SET50'],
 ['B', 'RCL', 1500, 41.5, 1, 'RD05pct', 'SET100'],
 ['B', 'ROJNA', 18000, 6.3, 1, '6L', 'SET'],
 ['B', 'SCC', 300, 374.0, 1, 'RD05pct', 'SET50'],
 ['B', 'STA', 2500, 28.0, 1, 'RD15%', 'SET100'],
 ['B', 'SYNEX', 1500, 27.5, 0, 'RD10pct', 'SET100'],
 ['B', 'TOP', 3000, 51.0, 1, 'ROUND', 'SET50'],
 ['B', 'TSE', 40000, 2.6, 1, '6L', 'SET'],
 ['B', 'TVO', 3600, 30.0, 1, 'DOS', 'SET100'],
 ['B', 'TYCN', 8000, 3.32, 1, 'RD10pct', 'SET'],
 ['S', 'CKP', 10000, 5.5, 0, '10pct', 'SET100'],
 ['S', 'EPG', 8000, 12.0, 0, '10pct', 'SET100'],
 ['S', 'GLOBAL', 2500, 21.0, 0, '5pct', 'SET50'],
 ['S', 'IMH', 3000, 20.7, 0, '15pct', 'mai'],
 ['S', 'IP', 1500, 23.2, 0, '15pct', 'mai'],
 ['S', 'IVL', 3000, 46.25, 1, 

In [12]:
for rcd in rcds:
    print(rcd)

['B', 'AEONTS', 1200, 180.0, 1, '3L', 'SET100']
['B', 'BGC', 10000, 10.0, 1, 'DOS', 'SET']
['B', 'NOBLE', 21000, 5.85, 1, 'RD15%', 'SET']
['B', 'PTT', 3000, 36.5, 1, 'RD05%', 'SET50']
['B', 'PTTGC', 1200, 54.25, 0, 'RD10pct', 'SET50']
['B', 'RATCH', 3000, 42.75, 1, 'RD05%', 'SET50']
['B', 'RCL', 1500, 41.5, 1, 'RD05pct', 'SET100']
['B', 'ROJNA', 18000, 6.3, 1, '6L', 'SET']
['B', 'SCC', 300, 374.0, 1, 'RD05pct', 'SET50']
['B', 'STA', 2500, 28.0, 1, 'RD15%', 'SET100']
['B', 'SYNEX', 1500, 27.5, 0, 'RD10pct', 'SET100']
['B', 'TOP', 3000, 51.0, 1, 'ROUND', 'SET50']
['B', 'TSE', 40000, 2.6, 1, '6L', 'SET']
['B', 'TVO', 3600, 30.0, 1, 'DOS', 'SET100']
['B', 'TYCN', 8000, 3.32, 1, 'RD10pct', 'SET']
['S', 'CKP', 10000, 5.5, 0, '10pct', 'SET100']
['S', 'EPG', 8000, 12.0, 0, '10pct', 'SET100']
['S', 'GLOBAL', 2500, 21.0, 0, '5pct', 'SET50']
['S', 'IMH', 3000, 20.7, 0, '15pct', 'mai']
['S', 'IP', 1500, 23.2, 0, '15pct', 'mai']
['S', 'IVL', 3000, 46.25, 1, '10pct', 'SET50']
['S', 'LPF', 10000, 14.

In [13]:
sql = """
INSERT INTO orders (trade, name, qty, price, active, reason, market)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active, reason, market)
VALUES (?, ?, ?, ?, ?, ?, ?)



In [14]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [15]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(df_orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=    id trade  ..., width=800)

In [16]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)